# Effective Bounds: A' and B'

LaTeX for the expressions below was automatically rendered by MathJax from the [PolyMath 15 test problem page](michaelnielsen.org/polymath1/index.php?title=Polymath15_test_problem) using a right-click to bring up a menu and choosing `Show math | as LaTeX commands`. It was then copied and pasted directly into the markdown source of this section. Presumably, this process is free of error.

It occurred to me that while translating mathematical notation into executable code, it would be handy to have the math juxtaposed with the code in the same document--someting Jupyter notebooks enable. Writing first in a notebook and automatically converting from notebook to source code, which Jupyter can also do, thus seemed like an efficient and reasonably error-free process.

The [test problem page](michaelnielsen.org/polymath1/index.php?title=Polymath15_test_problem) defines approximations $A'$ and $B'$ as follows.

$$A' = \frac{2}{8} \pi^{-s/2} \sqrt{2\pi} \exp( (\frac{s+4}{2}-\frac{1}{2}) \log \frac{s+4}{2} - \frac{s+4}{2}) \sum_{n=1}^N \frac{\exp(\frac{t}{16} \log^2 \frac{s+4}{2\pi n^2})}{n^s}$$


$$B' = \frac{2}{8} \pi^{-(1-s)/2} \sqrt{2\pi} \exp( (\frac{5-s}{2}-\frac{1}{2}) \log \frac{5-s}{2} - \frac{5-s}{2}) \sum_{n=1}^N \frac{\exp(\frac{t}{16} \log^2 \frac{5-s}{2\pi n^2})}{n^{1-s}}$$

$$B_0'= \frac{2}{8} \pi^{-(1-s)/2} \sqrt{2\pi} \exp( (\frac{5-s}{2}-\frac{1}{2}) \log \frac{5-s}{2} - \frac{5-s}{2}) \exp( \frac{t}{16} \log^2 \frac{5-s}{2\pi} )$$

It is easily seen from the mathematical notation that $B'(s) = A'(1-s),$ and that $B_0'$ is just $B'$ with $N=1.$ Thus it suffices to code only $A'$ in detail. 


In [1]:
using DBNUpperBound
using DBNUpperBound.Asymptotics

First coding the summand for $A'$

$$a_n(s) = \frac{\exp(\frac{t}{16} \log^2 \frac{s+4}{2\pi n^2})}{n^s}$$

In [2]:
function Aprime_a(t::T1, s::T2, n::Int) where {T1<:Real, T2<:Number}
    return bigexp((t/16)*log((s+4)/(2*π*n^2))^2 - s*log(n))
end

Aprime_a (generic function with 1 method)

(Bigexp will use multiprecision if single precision exponentiation yields 0 or $\pm \infty$.) 

Next, coding the multiplier of the $\sum_1^N$:
$$\mu(s) = \frac{2}{8} \pi^{-s/2} \sqrt{2\pi} \exp( (\frac{s+4}{2}-\frac{1}{2}) \log \frac{s+4}{2} - \frac{s+4}{2})$$

In [3]:
function Aprime_μ(t::T1, s::T2) where {T1<:Real, T2<:Number}
    return (2/8)*π^(-s/2)*√(2*π)*bigexp(((s+4)/2 - 1/2)*log((s+4)/2)-(s+4)/2)
end

Aprime_μ (generic function with 1 method)

Now,
$$ A' = \mu(s)\sum_1^N a_n(s).$$
Hence,

In [4]:
function Aprime(t::T1, s::T2, N::Int) where {T1<:Real, T2<:Number}
    asum = 0.0
    for n = 1:N
        asum += Aprime_a(t,s,n)
    end
    return Aprime_μ(t,s)*asum
end

Aprime (generic function with 1 method)

And since $B'(s) = A'(1-s)$ and $B0' = B'$ with $N=1$:

In [5]:
Bprime(t,s,N) = Aprime(t,1-s,N) # (This is legal function definition syntax in Julia)
B0prime(t,s) = Bprime(t,s,1)

B0prime (generic function with 1 method)

Comparison with the trusted Python implementation, `Ht_AFE_ADJ_AB`, serves as a quick initial test of the above. (The Python code is just cut and pasted from a terminal. It will not run in this notebook.) 

```
 >>> from mputility import *
 >>> z = 1000.0 + .4j
 >>> Ht_AFE_ADJ_AB(z,.4)
 >>> mpc(real='-5.1648957514753166742418230131433e-168', imag='3.13510483982411360761979246139316e-167')
```

In [6]:
z = 1000.0 + .4im
N = Nz(z)
t = .4
s = (1+im*z)/2
Aprime(t,s,N)+Bprime(t,s,N)

-5.16489575148183e-168 + 3.1351048398242333e-167im

At this point the notebook can be converted to a source code file using the `File|Download as|Julia(.jl)` menu. (Referring to the standard notebook UI, not the new [JupyterLab](https://channel9.msdn.com/Events/PyData/Seattle2017/BRK11).) A little manual editing will make the download suitable for inclusion in the `DBNUpperBound` package. (See [src/asymptotics/ABprime.jl](https://github.com/WilCrofter/DBNUpperBound.jl/blob/master/src/asymptotics/ABprime.jl).)